In [ ]:
#Conect my csv file to mango db#


In [ ]:
#Imort data first :
    
   # mongoimport --type csv -d meteorite -c meteorites_data --drop --headerline --file Meteorite_Landings_original_file.csv


In [1]:
# Working with the original CSV file and cleaning data to retrieve meteorite data from 1983 to 2023

from pymongo import MongoClient
from pprint import pprint

# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Specify the database and collection names
database_name = 'meteorite'
collection_name = 'meteorites_data'

# Assign the database and collection to variables
db = mongo[database_name]
collection = db[collection_name]

# Define the query to retrieve meteorites that fell between 1983 and 2023
query = {'year': {'$gte': 1983, '$lte': 2023}}

# Retrieve and print the first 5 meteorites that match the query
matching_meteorites = collection.find(query).limit(5)
print('First 5 meteorites that fell between 1983 and :')
for meteorite in matching_meteorites:
    pprint(meteorite)
    

First 5 meteorites that fell between 1983 and :
{'GeoLocation': '(13.66033, 28.96)',
 '_id': ObjectId('659f34b2c72de7d60a623e84'),
 'fall': 'Fell',
 'id': 447,
 'mass (g)': 700,
 'name': 'Al Zarnkh',
 'nametype': 'Valid',
 'recclass': 'LL5',
 'reclat': 13.66033,
 'reclong': 28.96,
 'year': 2001}
{'GeoLocation': '(45.82133, 6.01533)',
 '_id': ObjectId('659f34b2c72de7d60a623e88'),
 'fall': 'Fell',
 'id': 458,
 'mass (g)': 252,
 'name': 'Alby sur Chéran',
 'nametype': 'Valid',
 'recclass': 'Eucrite-mmict',
 'reclat': 45.82133,
 'reclong': 6.01533,
 'year': 2002}
{'GeoLocation': '(20.74575, 32.41275)',
 '_id': ObjectId('659f34b2c72de7d60a623e90'),
 'fall': 'Fell',
 'id': 48915,
 'mass (g)': 3950,
 'name': 'Almahata Sitta',
 'nametype': 'Valid',
 'recclass': 'Ureilite-an',
 'reclat': 20.74575,
 'reclong': 32.41275,
 'year': 2008}
{'GeoLocation': '(40.81056, 140.78556)',
 '_id': ObjectId('659f34b2c72de7d60a623e9c'),
 'fall': 'Fell',
 'id': 2313,
 'mass (g)': 320,
 'name': 'Aomori',
 'nametyp

In [2]:
# Saving clean data to csv file
import pandas as pd
# Retrieve meteorites that match the query and convert to a DataFrame
matching_meteorites = list(collection.find(query))
df = pd.DataFrame(matching_meteorites)

# Save the DataFrame as a CSV file
csv_file_path = 'meteorites1983-2023.csv'
df.to_csv(csv_file_path, index=False)

print(f'DataFrame saved as CSV file: {csv_file_path}')

DataFrame saved as CSV file: meteorites1983-2023.csv


In [7]:
# Looking for meteorite fell/found summary per year
# Saving retrieved data in CSV file

import csv
from pymongo import MongoClient

# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Specify the database and collection names
database_name = 'meteorite'
collection_name = 'meteorites_data'

# Assign the database and collection to variables
db = mongo[database_name]
collection = db[collection_name]

# Define the aggregation pipeline to count meteorites by year
pipeline = [
    {'$match': {'year': {'$gte': 1983, '$lte': 2023}}},  
    {'$group': {'_id': '$year', 'count': {'$sum': 1}}},  # Group by year and count the number of meteorites
    {'$sort': {'_id': 1}}
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Specify the path for the new CSV file
csv_file_path = 'meteorites_count_by_year.csv'

# Write the result to the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write header row
    csv_writer.writerow(['Year', 'Meteorites Count'])

    # Write data rows
    for group in result:
        year = group['_id']
        count = group['count']
        csv_writer.writerow([year, count])

print(f'CSV file "{csv_file_path}" created successfully.')


CSV file "meteorites_count_by_year.csv" created successfully.


In [6]:
import json
from pymongo import MongoClient
from pprint import pprint
from bson import ObjectId

# Custom JSON Encoder to handle ObjectId serialization
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super(JSONEncoder, self).default(o)

# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Specify the database and collection names
database_name = 'meteorite'
collection_name = 'meteorites_data'

# Assign the database and collection to variables
db = mongo[database_name]
collection = db[collection_name]

# Define the query to retrieve meteorites that fell between 1983 and 2023
query = {'year': {'$gte': 1983, '$lte': 2023}}

# Retrieve meteorites that match the query
matching_meteorites = list(collection.find(query))

# Specify the path for the new JSON file
json_file_path = 'meteorite 1983-2023.json'

# Write meteorite data to the JSON file using the custom encoder
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(matching_meteorites, json_file, indent=2, cls=JSONEncoder)

print(f'JSON file "{json_file_path}" created successfully.')



JSON file "meteorite 1983-2023.json" created successfully.
